In [1]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers.utils import export_to_gif
import os
import os.path as path

home = os.getenv('HOME')
p = StableDiffusionPipeline.from_single_file(path.join(home, 'civitai', 'animatediff', 'counterfeitV30_v30.safetensors'))

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler
from diffusers.utils import export_to_gif
import os
import os.path as path

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2")
model_id = "runwayml/stable-diffusion-v1-5"
pipe = AnimateDiffPipeline.from_pretrained(
  model_id,
  vae=p.vae.to(torch.float16),
  unet=p.unet.to(torch.float16),
  tokenizer=p.tokenizer,
  motion_adapter=adapter,
  torch_dtype=torch.float16,
  use_safetensors=True,
)
scheduler = DDIMScheduler.from_pretrained(
  model_id,
  subfolder="scheduler",
  clip_sample=False,
  timestep_spacing="linspace",
  beta_schedule="linear",
  steps_offset=1,
)
pipe.scheduler = scheduler

# pipe.enable_sequential_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

The config attributes {'transformer_layers_per_block': 1, 'attention_head_dim': 8, 'class_embed_type': None, 'addition_embed_type': None, 'addition_time_embed_dim': None, 'upcast_attention': None, 'projection_class_embeddings_input_dim': None} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
prompt = '(masterpiece, best quality),1girl, solo, flower, long hair, outdoors, letterboxed, school uniform, day, sky, looking up, short sleeves, parted lips, shirt, cloud, black hair, sunlight, white shirt, serafuku, upper body, from side, pink flower, blurry, brown hair, blue sky, depth of field'

generator = torch.Generator("cpu").manual_seed(2311942344)
output = pipe(
  prompt=prompt.strip(),
  num_frames=16,
  guidance_scale=9.0,
  num_inference_steps=30,
  generator=generator,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff.gif")

  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff.gif'

In [4]:
prompt = '(masterpiece, best quality),1girl with long white hair sitting in a field of green plants and flowers, her hand under her chin, warm lighting, white dress, blurry foreground'

generator = torch.Generator("cpu").manual_seed(1293666383)
output = pipe(
  prompt=prompt.strip(),
  num_frames=16,
  guidance_scale=9.0,
  num_inference_steps=30,
  generator=generator,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff2.gif")

  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff2.gif'

In [5]:
prompt = '((masterpiece,best quality)), 1girl, guinaifen, long kimono, long sleeves, fur trim, floral print, grin, petals, cherry blossoms, oil-paper umbrella, simple background,'
negative_prompt = 'bad-artist, (low quality, worst quality:1.4), (extra fingers, fewer fingers), Ugly Fingers, (deformed fingers:1.4), extra arms, extra legs, watermark, username, artist name,'

output = pipe(
  prompt=prompt.strip(),
  negative_prompt=negative_prompt.strip(),
  num_frames=16,
  guidance_scale=10.0,
  num_inference_steps=30,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff3.gif")

  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff3.gif'

In [6]:
prompt = '(masterpiece, best quality: 1.4), (1girl), long straight white hair, blue eyes, detailed face, sailor uniform, detailed clothing, combat pose, gust of wind, vivid colors, highest resolution, wallpaper, professional illustration, (realistic:1.2), intricate, highly detailed, landscape, dramatic lighting, cityscape view from harbor, daylight summertime'
negative_prompt = '(((deformed))), blurry, bad anatomy, disfigured, poorly drawn face, mutation, mutated, (extra_limb), (ugly), (poorly drawn hands), fused fingers, messy drawing, broken legs, (mutated hands and fingers:1.5), (long body :1.3), (mutation, poorly drawn :1.2), black-white, bad anatomy, liquid body, liquid tongue, disfigured, malformed, mutated, anatomical nonsense, error, malformed hands, long neck, blurred, low res, bad anatomy, bad proportions, bad shadow, uncoordinated body, unnatural body, huge breasts, huge haunch, huge thighs, huge calf, bad hands, fused hand, missing hand, disappearing arms, disappearing thigh, disappearing calf, disappearing legs, fused ears, bad ears, poorly drawn ears, extra ears, liquid ears, heavy ears, missing ears, fused animal ears, bad animal ears, poorly drawn animal ears, extra animal ears, liquid animal ears, heavy animal ears, missing animal ears, text, ui, error, missing fingers, missing limb, fused fingers, one hand with more than 5 fingers, one hand with less than 5 fingers, one hand with more than 5 digit, one hand with less than 5 digit, extra digit, fewer digits, fused digit, missing digit, bad digit, liquid digit, colorful tongue, black tongue, cropped, watermark, username, JPEG artifacts, signature, 3D, 3D game, 3D game scene, 3D character, malformed feet, extra feet, bad feet, poorly drawn feet, fused feet, missing feet, extra shoes, bad shoes, fused shoes, more than two shoes, poorly drawn shoes, bad gloves, poorly drawn gloves, fused gloves, bad hairs, poorly drawn hairs, fused hairs, big muscles, ugly, bad face, fused face, poorly drawn face, cloned face, big face, long face, bad eyes, fused eyes, poorly drawn eyes, extra eyes, malformed limbs, gross proportions, large breasts, elderly, extra limbs, watermark, logo, low resolution, medium resolution, deformed body, deformed limbs, ugly face, harelip, asymmetrical face, asymmetrical body, asymmetrical ears, ((multiple girls)), mutation, third leg, third arm, third hand, sixth finger, absurd fingers, merged limbs, amputated limbs, low lighting, dark, bleak color, overexposure, washed out color'

generator = torch.Generator("cpu").manual_seed(2378845630)
output = pipe(
  prompt=prompt.strip(),
  negative_prompt=negative_prompt.strip(),
  num_frames=16,
  guidance_scale=8.5,
  num_inference_steps=40,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff4.gif")

  0%|          | 0/40 [00:00<?, ?it/s]

'/tmp/animatediff4.gif'